In [335]:
import  dash 
from santiago_core import SANTIAGO_CORE
from dash import html ,dcc
import pandas as pd 
import plotly.express as px
import numpy as np
pd.set_option('display.max_columns', 500)

In [336]:
sa = SANTIAGO_CORE()

In [337]:
df_nc_ep = sa.df_nc.merge(sa.ep, how='left', left_on=['Cvendedor'], right_on=['Cod_Empleado'])

In [338]:
df_nc_ep.columns

Index(['Local_creacion', 'Desc_local', 'Dcompra_nvo', 'Nterminal_nvo',
       'Nsecuencia_nvo', 'Hora', 'Tipo_trx', 'Cautoriza', 'Estado', 'Usuario',
       'Cvendedor', 'Local_ant', 'Descr_local_ant', 'Dcompra_ant',
       'Nterminal_ant', 'Nsecuencia_ant', 'Cvendedor_ant', 'Cajero_apertura',
       'Linea', 'LiDescripcion', 'SKU', 'EAN', 'PDescripcion', 'Cmarca',
       'Tipo Producto', 'Nrutcomprador', 'Qcantidad', 'Costo_NC-Empleado',
       'Xtipificacion', 'Cod_Sucursal', 'Sucursal', 'Cod_Empleado',
       'Num_Documento', 'FecInicioContrato', 'Cod_Cargo', 'Cargo', 'Cod_Depto',
       'Departamento', 'Nombre_Superior', 'nombre_completo'],
      dtype='object')

Calcular y graficar cantidad de días entre ventas y NC


In [339]:

sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('ene', '01')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('feb', '02')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('mar', '03')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('abr', '04')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('may', '05')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('jun', '06')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('jul', '07')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('ago', '08')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('sep', '09')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('oct', '10')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('nov', '11')
sa.df_nc.Dcompra_ant = sa.df_nc.Dcompra_ant.str.replace('dic', '12')
sa.df_nc.Dcompra_ant = pd.to_datetime(sa.df_nc.Dcompra_ant, format='%d-%m-%Y') # TODO arreglar para español 




In [340]:
sa.df_nc["dif_dias"] = sa.df_nc.Dcompra_nvo - sa.df_nc.Dcompra_ant 

In [341]:
diferencia_dias = sa.df_nc

In [342]:
mes=0
mes_sig=0
mes_ant=0
while mes_sig <=2302:
    mes = mes + 1
    mes_sig= mes_sig + 30
    mes_ant = mes_sig - 30
    diferencia_dias.loc[(diferencia_dias["dif_dias"] >= f"{str(mes_ant)} days") & ((diferencia_dias["dif_dias"] <= f"{str(mes_sig)} days")), "mes"] = f"mes {mes}"
    


In [343]:
diferencia_dias = diferencia_dias.groupby("dif_dias")["Cautoriza"].nunique().reset_index()

In [344]:
diferencia_dias = diferencia_dias.loc[diferencia_dias.Cautoriza !=1 ]

In [345]:

dias_graf = px.bar(diferencia_dias, x="dif_dias", y="Cautoriza")
dias_graf.update_layout(xaxis_categoryorder = 'total descending')
dias_graf.show()



In [346]:
top_20_clientes_nc = sa.df_nc.groupby("Nrutcomprador")["Cautoriza"].nunique().reset_index().sort_values("Cautoriza",ascending=False)

In [347]:
top_20_clientes_nc.loc[~top_20_clientes_nc.Nrutcomprador.isin(["0","1","2"])].head(20)

,Nrutcomprador,Cautoriza
50024,1234567890,38
95301,52295256,28
38604,1110555895,24
105150,63317421,22
107486,66710306,21
15405,1020805672,20
87759,459687456,18
72764,37723241,16
77710,41328972,16
124524,90113482,16


In [348]:
nc_r= sa.df_nc.loc[sa.df_nc.Nrutcomprador == "0" ] 
ced_0 = nc_r.groupby(["Desc_local","Dcompra_nvo"])["Cvendedor"].nunique().sort_values(ascending=False).reset_index()
graf_ced_0 = px.bar(ced_0, x="Dcompra_nvo", y="Cvendedor", color="Desc_local" )
graf_ced_0.show()

In [349]:
nc_r= sa.df_nc.loc[sa.df_nc.Nrutcomprador == "1" ] 
ced_1 = nc_r.groupby(["Desc_local","Dcompra_nvo"])["Cvendedor"].nunique().sort_values(ascending=False).reset_index()
graf_ced_1 = px.bar(ced_1, x="Dcompra_nvo", y="Cvendedor", color="Desc_local" )
graf_ced_1.show()

In [350]:
nc_r= sa.df_nc.loc[sa.df_nc.Nrutcomprador == "2" ] 
ced_1 = nc_r.groupby(["Desc_local","Dcompra_nvo"])["Cvendedor"].nunique().sort_values(ascending=False).reset_index()
graf_ced_2 = px.bar(ced_1, x="Dcompra_nvo", y="Cvendedor", color="Desc_local" )
graf_ced_2.show()